In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [3]:

filename = "C:/Users/hp/Desktop/Basic RNN/wonder.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163419
Total Vocab:  58


In [9]:

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163319


In [10]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
filepath="C:/Users/hp/Desktop/Basic RNN/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
163319/163319 [==============================] - 930s 6ms/step - loss: 2.9762

Epoch 00001: loss improved from inf to 2.97618, saving model to C:/Users/hp/Desktop/Basic RNN/weights-improvement-01-2.9762.hdf5
Epoch 2/20
163319/163319 [==============================] - 1166s 7ms/step - loss: 2.7926

Epoch 00002: loss improved from 2.97618 to 2.79256, saving model to C:/Users/hp/Desktop/Basic RNN/weights-improvement-02-2.7926.hdf5
Epoch 3/20
163319/163319 [==============================] - 965s 6ms/step - loss: 2.7117

Epoch 00003: loss improved from 2.79256 to 2.71167, saving model to C:/Users/hp/Desktop/Basic RNN/weights-improvement-03-2.7117.hdf5
Epoch 4/20
163319/163319 [==============================] - 908s 6ms/step - loss: 2.6444

Epoch 00004: loss improved from 2.71167 to 2.64443, saving model to C:/Users/hp/Desktop/Basic RNN/weights-improvement-04-2.6444.hdf5
Epoch 5/20
163319/163319 [==============================] - 920s 6ms/step - loss: 2.5934

Epoch 00005: loss imp

In [14]:
filename = "C:/Users/hp/Desktop/Basic RNN/weights-improvement-20-2.0773.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [27]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Sky is:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Sky is:
"  as a
drawing of a muchness?'

'really, now you ask me,' said alice, very much confused, 'i don't
th "
ink it '
she said to herself, 'and thet sere toe would be oo toen io the soie.'

'what io toe toile 
Done.
